## Automatic differentiation and gradient tape, 自动微分

Automatic Differentiation(AD) 自动微分。

1. [Reverse-mode automatic differentiation: a tutorial](https://rufflewind.com/2016-12-30/reverse-mode-automatic-differentiation) 反向自动微分计算。
2. [Step-by-step example of reverse-mode automatic differentiation](https://stats.stackexchange.com/questions/224140/step-by-step-example-of-reverse-mode-automatic-differentiation)


我们以下面的式子为例：

$$
z = xy + sin(x) \tag{1}
$$

计算上式很简单，但是如果我想计算上式的微分该怎么计算那？

手动计算呀！但是如果是一个计算机程序，那我们应该怎么计算那？这个时候就需要考虑用到 AD 了。

AD 是基于以下实事的，即所有的程序操作都是基于一组基础的操作来实现的，如 +, *, 三角函数等。而再加上链式求导的法则，则可以让我们更好的利用这个性质。

### Forward-Mode AD

#### 表达式求值的程序计算

> Program A

```c
x = ? 
y = ?
a = x * y 
b = sin(x)
z = a + b 
```

对 (A) 中所有表达式都对 $t$ 进行求导。则有:

$$
\frac{\partial{x}}{\partial{t}} = ? \\
\frac{\partial{y}}{\partial{t}} = ? \\
\frac{\partial{a}}{\partial{t}} = y \frac{\partial{x}}{\partial{t}} + x  \frac{\partial{y}}{\partial{t}}\\
\frac{\partial{b}}{\partial{t}} = cos(x)\frac{\partial{x}}{\partial{t}} \\
\frac{\partial{z}}{\partial{t}} = \frac{\partial{a}}{\partial{t}} + \frac{\partial{b}}{\partial{t}}
\tag{F1}
$$

现在我们将 (F1) 写成计算机程序，用 ${dx, dy, \dots}$ 分别代替 ${\frac{\partial{x}}{\partial{t}}, \frac{\partial{y}}{\partial{t}}, \dots}$。 则有

> Program B

```c
dx = ?
dy = ?
da = ydx + xdy
db = cos(x)dx
dz = da + db
```

在 (F1) 中，
- 当我们用 $x = t$ 替换时就可以得到 $dx = 1, dy = 0$, 则最终我们就可以计算出： $\frac{\partial{z}}{\partial{x}}$

- 当我们用 $y = t$ 替换时就可以得到 $dx = 0, dy = 1$, 则最终我们就可以计算出： $\frac{\partial{z}}{\partial{y}}$


这样就计算出了程序 A 的导数了。

在实现时，我们只需要实现一个翻译程序，能够将程序 A 根据相应的规则翻译 成程序 B 即可。这些规则可以是：

```
c = a + b     =>    dc = da + db
c = a * b     =>    dc = b * da + a * db
c = sin(a)    =>    dc = cos(a) * da
c = a - b     =>    dc = da - db
c = a / b     =>    dc = da / b - a * db / b ** 2
```
等。

在翻译时，我只需要根据对应的规则进行替换即可。并且程序的顺序并不会改变，即：如果 K 在 L 之前求得，那么在求导的时候，K 也是在 L 之前的。所以这种也被称为 Forward-mode automatic differentiation. 前向自动微分。

FAD 的优点：

- 中间变量可以省略，节省内存。
- （Dual Number 二元数）有时间可以参考学习。

### Reverse-Mode Auto Diff

FAD 有一个缺点，就是当输入变量有 n 个时需要计算 n 次，即算法复杂度为  O(n)。哪上例有 x, y 两个自变量那我们就需要遍历两次来计算。

所以我们可用 RAD （逆序自动微分）。

RAD 也是复用链式求导的规则，即：

$$
\frac{dt}{dv}= \sum_{i}\frac{dt}{du_i} \frac{du_i}{dv}
$$

给定表达式，我们可生成一个计算图。假设我们的最终表达式值为 $z$, 其输入变量为 $w_i$, 从 $w_i$ 到 z 经过了 $w_p$ 节点，(即 $ z = g(w_p), w_p = f(w_i)$ ), 那我们可以求得输出变量对输入变量的导数为：

$$
\frac{dz}{dw_i} = \sum_{p \in parents(i)} \frac{dz}{dw_p} \frac{dw_p}{dw_i} \tag{2}
$$

这样，对于任意的中间节点，或者输入节点求导，我们只需要计算其父函数$w_p = f(w_i)$的基础求导公式即可。

下面我们就通过下的例子来说明逆向求导的过程。

假设我们表达式程序为：

$$
𝑤_1=𝑥_1\\
𝑤_2=𝑥_2\\
𝑤_3=𝑤_1𝑤_2\\
𝑤_4=sin(𝑤_1)\\
𝑤_5=𝑤_3+𝑤_4\\
𝑧=𝑤_5
$$

其计算图为: 

<img src="./reverse-mode-automatic-differentiation-graph.png" width="180px">

我们可以根据公式 (2) 来计算出所有结点上的梯度。

$$
\begin{align}
\frac{dz}{dw_5} &= z' \\
\frac{dz}{dw_4} &= \big(\frac{dz}{dw_5}\big) \frac{dw_5}{dw_4} \\
\frac{dz}{dw_3} &= \big(\frac{dz}{dw_5}\big) \frac{dw_5}{dw_3} \\
\frac{dz}{dw_2} &= \big(\frac{dz}{dw_3}\big) \frac{dw_3}{dw_2}  = \big(\frac{dz}{dw_5}\frac{dw_5}{dw_3}\big) \frac{dw_3}{dw_2} \\
\frac{dz}{dw_1} &= \frac{dz}{dw_4}\frac{dw_4}{dw_1} + \frac{dz}{dw_3}\frac{dw_3}{dw_1}  = \big(\frac{dz}{dw_5}\frac{dw_5}{dw_4}\big) \frac{dw_4}{dw_1} + \big(\frac{dz}{dw_5}\frac{dw_5}{dw_3}\big) \frac{dw_3}{dw_1}
\end{align}
$$

从上图可以看出，我们可以从底向上逆向求出所有导数，这样只需要计算一遍。我们可以将父结点的导数当作权重 weight, 即公式里用括号括起的部分。则在计算梯度时，即递归的用父结点权重与自身梯度相结合即可。


假设 $x_1 = 2, x_2 = 3$, 则有：

$$
𝑤_1=𝑥_1=2\\
𝑤_2=𝑥_2=3\\
𝑤_3=𝑤_1𝑤_2=6\\
𝑤_4=sin(𝑤_1) =0.9\\
𝑤_5=𝑤_3+𝑤_4=6.9\\
𝑧=𝑤_5=6.9\\
$$



首先有，

$$
\frac{dz}{dz} = 1
$$

又因为 $z = w_5$, 所以有 $\frac{dz}{dw_5} = 1$。又因为 $\frac{dw_5}{dw_3} = 1, \frac{dw_5}{dw_4} = 1$。所以中间的节点的 $w_3, w_4$ 的导数为：

$$
\frac{dz}{dw_3} = \frac{dz}{dw_5}\frac{dw_5}{dw_3} = 1 \times 1 \\
\frac{dz}{dw_4} = \frac{dz}{dw_5}\frac{dw_5}{dw_4} = 1 \times 1
$$

双因为 $w_3 = w_1 w_2$, 所以有 $\frac{dw_3}{dw_2} = w_1$, 所有：

$$
\frac{dz}{dw_2} = \frac{dz}{dw_3}\frac{dw_3}{dw_2} = 1 \times w_1 = w_1 
$$

根据表达式前向计算时，可以知道 

$$
\frac{dz}{dw_2} = w_1 = 2
$$

而对于输入变量 $w_1$, 其父函数有 $w_3, w_4$, 并且 $\frac{dw_3}{dw_1} = w_2, \frac{dw_4}{dw_1} = cos(w_1)$。所以们可以得到：

$$
\frac{dz}{dw_1} = \frac{dz}{dw_3}\frac{dw_3}{dw_1} + \frac{dz}{dw_4}\frac{dw_4}{dw_1} = w_2 + cos(w_1) 
$$

输入已知时，我们可以得到：

$$
\frac{dz}{dw_1} = w_2 + cos(w_1) = 3 + cos(2) = 2.58
$$

所以最终我们得到了:

$$
\frac{dz}{dx_1} = 2.58 \\
\frac{dz}{dx_2} = 2
$$

RAD 的过程就搞定了。

在上描述中，我们公考虑了标量，但是其实也是可以应用到向量和矩阵的。

[Step-by-step example of reverse-mode automatic differentiation](https://stats.stackexchange.com/questions/224140/step-by-step-example-of-reverse-mode-automatic-differentiation)

#### RAD 的算法实现

1. Naive 逆序树实现。
2. Gradient Tape 算法。

Naive 的方法是使用树的来实现，输入变量作为根结点。具体的参考 1.